In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 26.3 MB/s eta 0:00:00


In [2]:
import torch
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import re
import os
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score

In [3]:
sns.set_style("whitegrid")

In [4]:
# Check if cuda is available
assert torch.cuda.is_available(), "Cuda is not available, you cannot use the quantized version of the Mistral model without a GPU!"

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"All calculations will be performed on {device}")

All calculations will be performed on cuda


## Load Data

In [6]:
# Run this cell if you are using Google Colaboratory

from google.colab import drive
drive.mount('/content/drive')

your_path = "ITMO/RW'23/"

BASE_DIR = Path(f"/content/drive/MyDrive/{your_path}")
DATA_DIR = BASE_DIR.joinpath("data")  # Please upload the data_quality_estim.csv dataset to the data folder

FILE_NAME = "words_consistensy_wordset_dataset_agg.csv"

FILE_QUAL = DATA_DIR.joinpath(FILE_NAME)

Mounted at /content/drive


In [ ]:
# BASE_DIR = Path('Coherence_of_Words_4classes_Mistral_7B_Instruct_v20_4_bit.ipynb').resolve().parent.parent

# DATA_DIR_NAME = 'data/datasets'  # Local directory

# DATA_DIR = BASE_DIR.joinpath(DATA_DIR_NAME)
# FILE_NAME_QUAL = 'data_quality_estim_4classes.csv'

# FILE_QUAL = DATA_DIR.joinpath(FILE_NAME_QUAL)

In [7]:
data_quality = pd.read_csv(FILE_QUAL)
data_quality.head(10)

,INPUT:wordset,INPUT:dataset_name,OUTPUT:quality,homogenity,assessment
0,able amaze fill glad stop daily fantastic move...,amazon_food,"['rather_bad', 'bad', 'rather_bad']",0.7,-1
1,able amaze fill stop glad daily fantastic move...,amazon_food,"['bad', 'rather_good', 'rather_bad', 'rather_b...",0.5,-1
2,able amaze fill stop glad daily fantastic move...,amazon_food,"['bad', 'rather_bad', 'rather_bad']",0.7,-1
3,able amaze glad fill stop fantastic daily move...,amazon_food,"['bad', 'bad', 'bad']",1.0,-2
4,absolute church truth doug scripture symbol li...,20newsgroups,"['rather_good', 'bad', 'bad', 'rather_good', '...",0.6,1
5,absolute church truth mission nasa doug mars e...,20newsgroups,"['rather_bad', 'rather_bad', 'bad', 'rather_ba...",0.8,-1
6,absolute mission church nasa mars earth truth ...,20newsgroups,"['good', 'good', 'bad', 'good', 'good']",0.8,2
7,absolutely fast white rice service noodle terr...,amazon_food,"['rather_bad', 'bad', 'rather_bad', 'good', 'b...",0.5,-1
8,absolutely fast white super service customer r...,amazon_food,"['bad', 'good', 'rather_good', 'rather_good', ...",0.4,2
9,absolutely grain experience soon there's proce...,amazon_food,"['bad', 'bad', 'rather_bad']",0.7,-2


In [8]:
columns_map = {
    "INPUT:wordset": "task",
    "assessment": "agg_label"
}

data_quality = data_quality.rename(columns=columns_map)

data_quality.columns

Index(['task', 'INPUT:dataset_name', 'OUTPUT:quality', 'homogenity',
       'agg_label'],
      dtype='object')

In [9]:
print("The number of samples:", data_quality.shape[0])

The number of samples: 3861


In [11]:
data_quality["agg_label"].unique()

array([-1, -2,  1,  2,  0])

## Load Model

In [12]:
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

# Instruct-v0.2 Out of memory on inference Tesla T4
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

# Instruct-v0.2 4-bit quantization
# Qunatization config
quantization_config_4bit = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)

# Load model
model_4bit = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    quantization_config=quantization_config_4bit,
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

## Pipeline of the Inference

In [13]:
def generate_answer(prompt, model, device, temp, n_token):
    torch.cuda.empty_cache()
    encoded = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    model_input = encoded
    model_input = model_input.to(device)
    if device=='cpu':
        model.to(device)
    generated_ids = model.generate(
        **model_input, do_sample=True,
        max_new_tokens=n_token,
        temperature=temp,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.batch_decode(generated_ids)

    return decoded

In [14]:
def extract_substring(input_string):
    index = input_string.find("[/INST]")
    if index != -1:
        return input_string[index + len("[/INST]"):]
    else:
        return ""

In [16]:
def get_answer(
        task,
        model,
        device='cpu',
        temp=0.1,
        n_token=500,
        instruction='',
        sample='',
        print_system_prompt=False,
        save_system_prompt=False
        ):

    # Construct prompt
    if task == "classification":
        prompt = f"""<s> [INST] {instruction}\n{sample} [/INST]"""
    if print_system_prompt:
        print(prompt)
    # Generate answer
    answer = generate_answer(prompt, model, device, temp, n_token)
    # Extract the result
    if save_system_prompt:
        result = answer[0]
    else:
        result = extract_substring(answer[0])
    return result.replace("\n", " ")

In [17]:
def get_prompt(prompt_type, sample):
    if prompt_type == "P1":
        system_prompt = ("You are a helpful assistant evaluating the top words of a topic model output for a given topic. "
                "You will recieve a list of words. Please rate the following words on a scale from -2 to 2 by answering "
                "on the question: 'Is it possible to determine a common topic for the presented word set or at least "
                "for the most part of the set?'. Use the criteria: '-2' when words seem to be unconnected, "
                "'-1' if the amount of irrelevant words is high to determine a topic or there is a mixture "
                "of topics, '1' if some words are too common or out of topic, '2' if words have a strong "
                "connection between them, '0' if it's hard for you to answer on the question. "
                "Print the answer in the form: Answer: [rating]. Print only answer without explanation.\n")
        sample = 'Words: ' + sample
    elif prompt_type == "P2":
        system_prompt = ("You are a useful assistant who evaluates the coherence of words. "
                 "You will receive a list of words, please determine which class the given "
                 "list of words belongs to by answering the question: 'Is it possible to determine "
                 "a common topic for the presented word set or at least for the most part of the set?'.\n"
                 "Classification rules:\nyes - if words have a strong connection between them\n"
                 "rather yes - if some words are too common or out of topic\n"
                 "rather no - if the amount of irrelevant words is high to determine a topic or there is a mixture of topics\n"
                 "no - when words seem to be unconnected\n"
                 "neutral - if it is hard for you to answer on the question.\nPrint only class without explanation and additional information.\n")

        sample = 'Words: ' + sample + "\nClass:"

    return system_prompt, sample

In [23]:
def get_results(data, model, task, prompt_type, filename, num_iter, device='cpu', temp=0.1, n_token=500,
                shuffle_data=False, remove_old_file=True, print_info=False):
    # Create file
    # data_dir = BASE_DIR.joinpath('matveeva/results')
    data_dir = BASE_DIR.joinpath('data/results')
    res_file = data_dir.joinpath(filename)

    # Remove old file
    if remove_old_file:
        if os.path.exists(res_file):
            os.remove(res_file)

    # Shuffle data
    if shuffle_data:
        data = data.sample(frac=1, random_state=42)

    for i in tqdm(range(num_iter)):
        sample = data.iloc[i]["task"]
        if task == "classification":
            system_prompt, sample = get_prompt(prompt_type, sample)

            res_true = data.iloc[i]["agg_label"]
            res_model = get_answer(
                task=task,
                model=model,
                device=device,
                temp=temp,
                n_token=n_token,
                instruction=system_prompt,
                sample=sample,
                print_system_prompt=print_info
            )
            if print_info:
                print(res_model)

            with open(res_file, 'a') as file:
                file.write(f"{data.iloc[i]['task']}\t{res_true}\t{res_model}\n")

## Task 1. Evaluating the Coherence of Provided Words (Classification Task, 5 classes)

## Prompt Engineering


### Type I. Zero-shot Chain-of-Thought

In [ ]:
system_prompt = ("You are a helpful assistant evaluating the top words of a topic model output for a given topic. "
                "You will recieve a list of words. Please rate the following words on a scale from -2 to 2 by answering "
                "on the question: 'Is it possible to determine a common topic for the presented word set or at least "
                "for the most part of the set?'. Use the criteria: '-2' when words seem to be unconnected, "
                "'-1' if the amount of irrelevant words is high to determine a topic or there is a mixture "
                "of topics, '1' if some words are too common or out of topic, '2' if words have a strong "
                "connection between them, '0' if it's hard for you to answer on the question. "
                "Print the answer in the form: Answer: [rating]. Print only answer without explanation.\n")

sample = data_quality.iloc[1]["task"]

get_answer(
    task="classification",
    model=model_4bit,
    device=device,
    instruction=system_prompt,
    sample=sample,
    n_token=10,
    print_system_prompt=True
)

<s> [INST] You are a helpful assistant evaluating the top words of a topic model output for a given topic. You will recieve a list of words. Please rate the following words on a scale from -2 to 2 by answering on the question: 'Is it possible to determine a common topic for the presented word set or at least for the most part of the set?'. Use the criteria: '-2' when words seem to be unconnected, '-1' if the amount of irrelevant words is high to determine a topic or there is a mixture of topics, '1' if some words are too common or out of topic, '2' if words have a strong connection between them, '0' if it's hard for you to answer on the question. Print the answer in the form: Answer: [rating]. Print only answer without explanation.

able amaze fill stop glad daily fantastic move plenty prime empty job subtle strongly litter [/INST]


' Answer: 1.</s>'

**Conclusion:** The model understands what is required of her. This prompt can be used for further research.

### Type II. Zero-shot Chain-of-Thought

In [21]:
system_prompt = ("You are a useful assistant who evaluates the coherence of words. "
                 "You will receive a list of words, please determine which class the given "
                 "list of words belongs to by answering the question: 'Is it possible to determine "
                 "a common topic for the presented word set or at least for the most part of the set?'.\n"
                 "Classification rules:\nyes - if words have a strong connection between them\n"
                 "rather yes - if some words are too common or out of topic\n"
                 "rather no - if the amount of irrelevant words is high to determine a topic or there is a mixture of topics\n"
                 "no - when words seem to be unconnected\n"
                 "neutral - if it is hard for you to answer on the question.\nPrint only class without explanation and additional information.\n")

sample = 'Words: ' + data_quality.iloc[0]["task"] + "\nClass:"

get_answer(
    task="classification",
    model=model_4bit,
    device=device,
    instruction=system_prompt,
    sample=sample,
    n_token=10,
    print_system_prompt=True
)

<s> [INST] You are a useful assistant who evaluates the coherence of words. You will receive a list of words, please determine which class the given list of words belongs to by answering the question: 'Is it possible to determine a common topic for the presented word set or at least for the most part of the set?'.
Classification rules:
yes - if words have a strong connection between them
rather yes - if some words are too common or out of topic
rather no - if the amount of irrelevant words is high to determine a topic or there is a mixture of topics
no - when words seem to be unconnected
neutral - if it is hard for you to answer on the question.
Print only class without explanation and additional information.

Words: able amaze fill glad stop daily fantastic move prime plenty empty subtle job strongly litter
Class: [/INST]


' neutral</s>'

## Inference on the Entire Dataset

### Prompt I

In [ ]:
RES_FILE_NAME = "text_coherence_mistral_7b_4_bit_5classes_P1.csv"

get_results(
    data=data_quality,
    model=model_4bit,
    task='classification',
    prompt_type='P1',
    filename=RES_FILE_NAME,
    num_iter=data_quality.shape[0],
    device=device,
    temp=0.1,
    n_token=10,
    shuffle_data=False,
    remove_old_file=True,
    print_info=False
)

100%|██████████| 3861/3861 [2:39:11<00:00,  2.47s/it]



### Prompt II

In [26]:
RES_FILE_NAME = "text_coherence_mistral_7b_4_bit_5classes_P2.csv"

get_results(
    data=data_quality,
    model=model_4bit,
    task='classification',
    prompt_type='P2',
    filename=RES_FILE_NAME,
    num_iter=data_quality.shape[0],
    device=device,
    temp=0.1,
    n_token=10,
    shuffle_data=False,
    remove_old_file=True,
    print_info=False
)

100%|██████████| 3861/3861 [2:08:17<00:00,  1.99s/it]
